# Step 1: Collecting Data

In [21]:
import pandas as pd
import numpy as np
import praw
import tweepy as tw
import datetime

### Keywords found using GPT 4 (LLM) from across news articles
##### The prompt engineered by us was to select keywords across 50 articles was:
##### 

In [8]:
KeyWords = ["Armenia"]

##### Can't use Twitter API as it is Paid, $100 per month thanks to Elon Musk :)

In [12]:
# Twitter API Credentials
API_KEY = 'I0YJ2KCkpX5zMLBsiXWg8oIpm'
API_SECRET_KEY = 'B0smJRkFAcX702HTqzk2SjmlOn3ODze5HepNqnv15xprtyLIeO'
ACCESS_TOKEN = '1714199019898241024-kc26PuFJ9YOlgQddlxTuvLcRpACqKT'
ACCESS_TOKEN_SECRET = '1tPZmQtSvc1IiY5tnUIEFKBS8IuXZoMKU9PdXuSsyLZso'

# Authenticate with Tweepy
auth = tw.OAuth1UserHandler(
    consumer_key=API_KEY,
    consumer_secret=API_SECRET_KEY,
    access_token=ACCESS_TOKEN,
    access_token_secret=ACCESS_TOKEN_SECRET
)

api = tw.API(auth)

In [ ]:
numberOfTweets = 2
tweets = []

for tweet in tw.Cursor(api.search_tweets, q=KeyWords, lang="en").items(numberOfTweets):
    tweets.append(tweet)

In [ ]:
data = {'Tweet': [], 'Time': []}
for tweet in tweets:
    data['Tweet'].append(tweet.text)
    data['Time'].append(tweet.created_at)

df = pd.DataFrame(data)

##### Reddit

In [98]:
# Reddit App Credentials
CLIENT_ID = 'mPppFSzc3O_wFZzkWt61LA'
CLIENT_SECRET = 'eObtVXwrmq68OS89P5fWWADOzUWC9w'
USER_AGENT = 'web app by /u/Confident-Common3839'

# Authenticate with PRAW
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT
)

# Search for posts containing a specific keyword across all of Reddit
query = KeyWords
numberOfRedditPosts = 1000
df_reddit = pd.DataFrame(columns=['Date', 'Topic', 'Sub Topic'])  # Create an empty DataFrame with columns

for post in reddit.subreddit('all').search(query, limit=numberOfRedditPosts):
    date = datetime.datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d')
    new_row = pd.DataFrame({'Date': date, 'Topic': post.title, 'Sub Topic': post.selftext}, index=[0])
    df_reddit = pd.concat([df_reddit, new_row], ignore_index=True)

In [99]:
print(df_reddit.count())
df_reddit.head(5)

Date         238
Topic        238
Sub Topic    238
dtype: int64


,Date,Topic,Sub Topic
0,2023-09-30,Armenia will be invaded if West does not stand...,
1,2023-07-05,Armenia Denied EU Military Aid,
2,2023-09-26,"""Don't believe Armenia"", Azerbaijan(2020)",
3,2023-07-20,"Turkophobia in Armenia, situation getting wors...","They attacking them, insulting, damaging prope..."
4,2023-09-19,Map of the recent developments in the Armenia ...,


##### New York Times

In [90]:
import requests

# NYT API Key
API_KEY = '9NUbrFPCCZLBA292z2H4IGSJprHb7AGT'

# Base URL for Article Search API
BASE_URL = "https://api.nytimes.com/svc/search/v2/articlesearch.json"

df_NYT = pd.DataFrame(columns=['Date', 'Topic'])  # Create an empty DataFrame with columns


for i in range(0, 10): #scrape 10 pages
    params = {
    'q': KeyWords,  # Search terms
    'api-key': API_KEY,
    'page': i }
    response = requests.get(BASE_URL, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        docs = response.json().get('response').get('docs')
        for doc in docs:
            print(doc['headline']['main'])  # Print the headline of the article
            print(doc['web_url'])           # Print the URL of the article
            # print(doc['pub_date'])          Print the publish date of the article
            #convert the date to a date only format
            date = datetime.datetime.strptime(doc['pub_date'], '%Y-%m-%dT%H:%M:%S%z').strftime('%Y-%m-%d')
            print(date)
            #Append to the dataframe
            new_row = pd.DataFrame({'Date': [date], 'Topic': [doc['headline']['main']]})
            df_NYT = pd.concat([df_NYT, new_row], ignore_index=True)
    else:
        print(f"Error {response.status_code}: Unable to fetch data from The New York Times API")

Armenia: Cast Adrift in a Tough Neighborhood
https://www.nytimes.com/2023/09/29/world/europe/armenia-azerbaijan-nagorno-karabakh.html
2023-09-29
Azerbaijan Attacks Armenian Enclave of Nagorno-Karabakh
https://www.nytimes.com/video/world/asia/100000009094569/azerbaijan-armenia-nagorno-karabakh.html
2023-09-20
After Surrender in Nagorno-Karabakh, Leaders Discuss Armenians’ Fate
https://www.nytimes.com/2023/09/21/world/europe/armenia-azerbaijan-nagorno-karabakh.html
2023-09-21
Azerbaijan Reclaims Armenian Enclave, Shifting Region’s Political Dynamics
https://www.nytimes.com/2023/09/20/world/europe/azerbaijan-armenia-cease-fire.html
2023-09-20
Refugees Flee to Armenia as Breakaway Enclave Comes Under Azerbaijan’s Control
https://www.nytimes.com/2023/09/24/world/europe/armenians-nagorno-karabakh-azerbaijan.html
2023-09-24
Haley Hits Trump and Ramaswamy Over Israel Remarks: ‘That’s Not What We Need’
https://www.nytimes.com/2023/10/13/us/politics/nikki-haley-trump-ramaswamy-israel.html
2023-1

In [94]:
print(df_NYT.count())
df_NYT.head(5)

Date     50
Topic    50
dtype: int64


,Date,Topic
0,2023-09-29,Armenia: Cast Adrift in a Tough Neighborhood
1,2023-09-20,Azerbaijan Attacks Armenian Enclave of Nagorno...
2,2023-09-21,"After Surrender in Nagorno-Karabakh, Leaders D..."
3,2023-09-20,"Azerbaijan Reclaims Armenian Enclave, Shifting..."
4,2023-09-24,Refugees Flee to Armenia as Breakaway Enclave ...


##### Using LLMs to detect if related enough to our topic

In [ ]:
# We will use meta llama 2 - 70b to detect if the topic in the data frame is related to Armenia Conflict/war with Azerbaijan
